# Project 1.1: Object Detection in Urban Environments
## Part 1: Exploring the dataset — Basics
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Self-Driving Car Engineer Nanodegree programme offered at Udacity.

## Objectives

* Fetch the [Waymo Open Dataset](https://waymo.com/open/);
* Visualise a few images and their bounding boxes.

## 1. Introduction

In [1]:
### Importing the required modules

In [2]:
import glob
import logging
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os

In [3]:
### Installing the Waymo Open Dataset API and dependencies
!pip3 install --quiet --upgrade pip
!pip3 install --quiet numpy==1.21.6
!pip3 install --quiet waymo-open-dataset-tf-2-6-0==1.4.9

     |████████████████████████████████| 2.0 MB 27.8 MB/s 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.4/564.4 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 80.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
jax 0.3.17 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.0.7 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.


In [18]:
#### Importing the Waymo Open Dataset API
import google.protobuf
import tensorflow as tf
import waymo_open_dataset
from waymo_open_dataset import dataset_pb2 as open_dataset

In [5]:
### Setting the environment variables

In [6]:
ENV_COLAB = True                # True if running in Google Colab instance

In [7]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [8]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

In [9]:
### Creating subdirectories (if not exists)
os.makedirs(DIR_OUT, exist_ok=True)

### 1.1. The Waymo Open Dataset

The [Waymo Open Dataset for Perception](https://waymo.com/open/data/perception) [1] is a publicly available dataset consisting of high-resolution sensor data and labels for 2,030 driving scenes.

A nice summary of the `waymo_open_dataset` API and data attributes can be found on [this repository](https://github.com/Jossome/Waymo-open-dataset-document) by GitHub user Jossome [2] or in the TensorFlow Datasets Catalog [here](https://www.tensorflow.org/datasets/catalog/waymo_open_dataset).

#### Getting familiar with classes

The `Box` attribute has a label that can be one of five possible object classes defined in [`label.proto`](https://github.com/waymo-research/waymo-open-dataset/blob/master/waymo_open_dataset/label.proto#L63):
```
TYPE_UNKNOWN = 0
TYPE_VEHICLE = 1
TYPE_PEDESTRIAN = 2
TYPE_SIGN = 3
TYPE_CYCLIST = 4
```

#### Closer look at bounding boxes data

The Waymo Open Dataset is a multi-sensor, multi-modal dataset. Every frame in the dataset consists of data captured from multiple camera views surrounding the Waymo ego vehicle. The data from any one camera can be accessed with `frame.camera_labels` access chain, followed by any one of the following `CameraName` enums defined in [`dataset.proto`](ataset/blob/master/waymo_open_dataset/dataset.proto#L48):
```
UNKNOWN = 0
FRONT = 1
FRONT_LEFT = 2
FRONT_RIGHT = 3
SIDE_LEFT = 4
SIDE_RIGHT = 5
```

To access the front camera's bounding box data, for example, the following access chain is used:
```
boxes = frame.camera_labels[0]
```
which will return a `waymo_open_dataset.dataset_pb2.CameraLabels` instance.

Note that we have to access each camera's data by its index. In the Waymo Open Dataset v1.2, the camera indices are slightly different from the above Enum definitions. That is, there is no `UNKNOWN` camera label, and the camera indices starting with `FRONT` begin at index `0`.

To access the first `Box` instance from the `FRONT` camera, we can use
```
box = frame.camera_labels[0].labels[0]
```
which will return a single `waymo_open_dataset.label_pb2.Label` instance. Each `Label` instance has several important attributes:

* `box`: the upright bounding box coordinates and dimensions in vehicle frame;
* `type`: the class label (int) of the object (e.g., `TYPE_VEHICLE`, `TYPE_PEDESTRIAN`);
* `id`: the unique identifier assigned to this object;
* `detection_difficulty_level`: the difficulty level for detection;
* `tracking_difficulty_level`: the difficulty level for tracking.

More on the specifics of the attribute values can be found in the [`label.proto`](https://github.com/waymo-research/waymo-open-dataset/blob/master/waymo_open_dataset/label.proto) file and on the [`labeling_specifications.md`](https://github.com/waymo-research/waymo-open-dataset/blob/master/docs/labeling_specifications.md) document.

## 2. Programming Task

### 2.1. Fetching the Waymo Open Dataset

In [10]:
### Path to store all downloaded `.tfrecord` training files
DIR_TRAIN = os.path.join(DIR_SRC, 'train')

In [11]:
### Create subdirectory if not already exists
os.makedirs(DIR_TRAIN, exist_ok=True)

#### Using `gsutil`

In [12]:
### Authenticating the Google user account authorized to access 
#   the Waymo Open Dataset via GCS / gsutil
from google.colab import auth
auth.authenticate_user()

In [13]:
### Setting the gsutil variables
bucket_name = 'waymo_open_dataset_v_1_2_0_individual_files/training/'
file_name = 'segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord'

In [14]:
### Downloading the specified `.tfrecord` fle via GCS gsutil
#   Note: GCS Public or Authenticated URLs do *not* work (`Permission denied`)
!gsutil cp "{'gs://' + bucket_name + file_name}" "{DIR_TRAIN}"

Copying gs://waymo_open_dataset_v_1_2_0_individual_files/training/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord...
\ [1 files][940.1 MiB/940.1 MiB]   73.4 MiB/s                                   
Operation completed over 1 objects/940.1 MiB.                                    


In [15]:
### List of all `.tfrecord` files in training subdirectory
files_to_convert = [f for f in os.listdir(DIR_TRAIN) if (
                    f.endswith('.tfrecord'))]
files_to_convert

['segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord']

In [16]:
### Alternatively, we can use glob to get the absolute paths
file_paths_to_convert = glob.glob('{}//*.tfrecord'.format(DIR_TRAIN))
file_paths_to_convert

['/content/data/train/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord']

#### Getting the first frame

In [19]:
### Trying with the example record fetched from GCS
FILE_PATH_WAYMO = file_paths_to_convert[0]
example_set = tf.data.TFRecordDataset(FILE_PATH_WAYMO, compression_type='')
# Return the first frame in the record
for i, data in enumerate(example_set):
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    frame_nr = i+1
    break

The Waymo `Frame` object is a container for all attributes collected at a specific context (point in time within a driving scene).

In [20]:
type(frame)

waymo_open_dataset.dataset_pb2.Frame

The tree structure listing the attributes associated with a `Frame` object are visualised nicely on the [Waymo Open Dataset Document](https://github.com/Jossome/Waymo-open-dataset-document) repository.

In [21]:
frame.context

name: "1005081002024129653_5313_150_5333_150"
camera_calibrations {
  name: FRONT
  intrinsic: 2083.091212133254
  intrinsic: 2083.091212133254
  intrinsic: 957.2938286685071
  intrinsic: 650.5697927719348
  intrinsic: 0.04067236637270731
  intrinsic: -0.3374271466716414
  intrinsic: 0.0016273829099200004
  intrinsic: -0.0007879327563938157
  intrinsic: 0.0
  extrinsic {
    transform: 0.9999151800844592
    transform: -0.008280529275085654
    transform: -0.010053132426658727
    transform: 1.5444145042510942
    transform: 0.008380895965622895
    transform: 0.9999150476776223
    transform: 0.009982885888937929
    transform: -0.022877347388980857
    transform: 0.009969614810858722
    transform: -0.010066293398396434
    transform: 0.9998996332221252
    transform: 2.115953541712884
    transform: 0.0
    transform: 0.0
    transform: 0.0
    transform: 1.0
  }
  width: 1920
  height: 1280
  rolling_shutter_direction: RIGHT_TO_LEFT
}
camera_calibrations {
  name: FRONT_LEFT
  intr

## Credits

This assignment was prepared by Thomas Hossler and Michael Virgo et al., Winter 2021 (link [here](https://github.com/udacity/nd013-c1-vision-starter)).

References
* [1] Sun, Pei, et al. "Scalability in Perception for Autonomous Driving: Waymo Open Dataset." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR). 2020. https://waymo.com/open.


Helpful resources:
* [Exercise 1.1.3: Creating `TFRecord`s by Jonathan L. Moran | GitHub](https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/1-1-Object-Detection-2D/1-Computer-Vision/Exercises/1-1-3-Creating-TF-Records/2022-08-03-Creating-TF-Records.ipynb)